# Many-body potentials

``matscipy`` implements support for generic manybody potentials. The class {py:class}`matscipy.calculators.manybody.Manybody` implements the functional form

```{math}
  U
  =
  \frac{1}{2}
  \sum_{\substack{ij\\ i\neq j}}
  U_2(r^2_{ij}) + U_\text{m}(r^2_{ij}, \xi_{ij})
```

with

```{math}
  \xi_{ij} 
  = 
  \sum_{\substack{k\\ k\neq i,j}} 
  \Xi(r^2_{ij}, r^2_{ik}, r^2_{jk})
```

as described, e.g. by [Müser et al.](https://doi.org/10.1080/23746149.2022.2093129) and [Grießer et al.](https://doi.org/10.48550/arXiv.2302.08754). On top of energies and forces, the calculator can compute second derivatives (with respect to positions and strain degrees of freedom). Explicit functional forms of {math}`U_2`, {math}`U_\text{m}` and {math}`\Xi` are implemented for a number of potential.

## Kumagai potential

The following example computes the elastic constants of a small representation of amorphous silicon using the potential by [Kumagai et al.](https://doi.org/10.1016/j.commatsci.2006.07.013). We first load the amorphous structure.

In [ ]:
from ase.io import read

def interactive_view(system):
    from ase.visualize import view
    # Interactive view of the lattice
    v = view(system, viewer='ngl')

    # Resize widget
    v.view._remote_call("setSize", target="Widget", args=["300px", "300px"])
    v.view.center()
    return v

a = read('../../tests/aSi.cfg')
a.symbols[:] = 'Si'

interactive_view(a)

Now we setup the calculator with the Kumagai potential and its parametrization.

In [ ]:
from matscipy.calculators.manybody import Manybody
from matscipy.calculators.manybody.explicit_forms import Kumagai
from matscipy.calculators.manybody.explicit_forms.kumagai import Kumagai_Comp_Mat_Sci_39_Si
from IPython.display import display, Markdown

a.calc = Manybody(**Kumagai(Kumagai_Comp_Mat_Sci_39_Si))

display(Markdown(f'Cohesive energy = {a.get_potential_energy() / len(a):.2f} eV/atom'))

We can also compute the Born elastic constants, i.e. the affine elastic constants:

In [ ]:
import numpy as np
from ase.units import GPa
from matscipy.elasticity import elastic_moduli, full_3x3x3x3_to_Voigt_6x6

# Born elastic constants (without nonaffine displacements)
C = a.calc.get_property('born_constants', a)

# Get isotropic elastic moduli
E, nu, G, B, K = elastic_moduli(full_3x3x3x3_to_Voigt_6x6(C))

display(Markdown(f"Young's modulus = {E.mean() / GPa:.1f} GPa"))
display(Markdown(f"Poisson number = {(nu + np.eye(3)).sum()/6:.3f}"))

## Other many-body potentials

To runs the same code with [Stillinger-Weber](https://doi.org/10.1103/PhysRevB.31.5262), replace the calculator by

In [ ]:
from matscipy.calculators.manybody.explicit_forms import StillingerWeber
from matscipy.calculators.manybody.explicit_forms.stillinger_weber import Stillinger_Weber_PRB_31_5262_Si

a.calc = Manybody(**StillingerWeber(Stillinger_Weber_PRB_31_5262_Si))

display(Markdown(f'Cohesive energy = {a.get_potential_energy() / len(a):.2f} eV/atom'))

Other examples of potentials that are implemented are [Tersoff](https://doi.org/10.1103/PhysRevB.39.5566), [Brenner](https://doi.org/10.1103/PhysRevB.42.9458) (without the lookup tables), [Erhart-Albe](https://doi.org/10.1103/PhysRevB.71.035211) and others. Second derivatives are supported for all of these.